In [1]:
import numpy as np
import pandas as pd
import math
from typing import Tuple

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [2]:
def EvenDifference(points):
    """
    Calculates the finite difference table for evenly spaced nodes.
    
    Args:
        points (list of tuples): A list of (x, y) data points, sorted by x.

    Returns:
        pandas.DataFrame: The full difference table.
    """
    x_values = [p[0] for p in points]
    y_values = [p[1] for p in points]
    n = len(y_values)

    # Internal calculation table
    diff_calc_table = np.full((n, n), np.nan)
    diff_calc_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_calc_table[i, j] = diff_calc_table[i+1, j-1] - diff_calc_table[i, j-1]

    # Format the output DataFrame
    data = {
        'x_i': x_values,
        'y_i': y_values
    }
    for j in range(1, n):
        # Pad with NaN to maintain table shape
        col_name = f'Order {j}'
        col_data = np.full(n, np.nan)
        col_data[:(n-j)] = diff_calc_table[:(n-j), j]
        data[col_name] = col_data
    
    df = pd.DataFrame(data)
    return df


# Bessel

## Algorithm

0. Conditions
* Used for **evenly spaced nodes**: $x_k = x_0 + kh$, $k \in \mathbb{Z}$.
* Requires an **even number of nodes** (which means an odd degree $n$).
* The formula is centered *between* two nodes, $x_0$ and $x_1$.

1. Finite Difference Table

Given $n+1$ nodes (where $n$ is odd). The formula is centered between $x_0$ and $x_1$.
A standard finite difference table is computed.

*(This is performed by the `EvenDifference(points)` function, which can be found in `Week6-Interpolation/pp2_Newton/pp2b_NewtonFixedGap.ipynb`)*

2. Coefficient Selection

The coefficients $C_i$ are selected from the difference table horizontally between the $y_0$ and $y_1$ rows:
* $C_0 = \frac{y_0 + y_1}{2}$
* $C_1 = \Delta y_0$
* $C_2 = \frac{\Delta^2 y_{-1} + \Delta^2 y_0}{2}$
* $C_3 = \Delta^3 y_{-1}$
* $C_4 = \frac{\Delta^4 y_{-2} + \Delta^4 y_{-1}}{2}$
* $C_5 = \Delta^5 y_{-2}$
* ...
* **General form:**
    * $C_{2k} = \frac{\Delta^{2k} y_{-k} + \Delta^{2k} y_{-(k-1)}}{2}$
    * $C_{2k+1} = \Delta^{2k+1} y_{-k}$

3. Polynomial Construction in $t$

The polynomial is constructed in the variable $t = \frac{x - x_0}{h}$. The formula is "centered" at $t=0.5$.
$P(t) = \sum_{i=0}^{n} D_i B_i(t)$

1.  **Calculate Main Coefficients $D_i$:**
    $D_i = \frac{C_i}{i!}$

2.  **Calculate Basis Polynomials $B_i(t)$:**
    * $B_0(t) = 1$
    * $B_1(t) = (t - \frac{1}{2})$
    * $B_2(t) = t(t - 1)$
    * $B_3(t) = B_2(t) \cdot (t - \frac{1}{2})$
    * $B_4(t) = B_2(t) \cdot (t + 1)(t - 2)$
    * $B_5(t) = B_4(t) \cdot (t - \frac{1}{2})$
    * ...
    * **Recursive form:**
        * $B_{2k}(t) = B_{2k-2}(t) \cdot (t + k - 1)(t - k)$
        * $B_{2k+1}(t) = B_{2k}(t) \cdot (t - \frac{1}{2})$

3.  **Compute Total Polynomial $P(t)$:**
    $P(t) = \sum_{i=0}^{n} N_i(t) = \sum_{i=0}^{n} D_i B_i(t)$
    * The coefficients of each $N_i(t)$ are summed by degree to find the final coefficients of $P(t) = a_0 + a_1 t + \dots + a_n t^n$.

4.  **Output:** Return the intermediate steps table and the final coefficient list $a_k$ for $P(t)$.

In [3]:
def bessel_interpolation(points, x0_index):
    """
    Constructs the Bessel interpolation polynomial in the variable t = (x - x0) / h.
    
    Args:
        points (list of tuples): List of (x, y) data points. Must be evenly spaced
                                 and have an even number of points.
        x0_index (int): The index of the first central node (x0).

    Returns:
        step_pd (pd.DataFrame): DataFrame showing the intermediate steps.
        coeff_pd (pd.DataFrame): DataFrame of the final polynomial coefficients for P(t).
    """
    
    n = len(points) - 1
    if (n + 1) % 2 != 0:
        raise ValueError("Bessel's formula requires an even number of nodes (odd degree n).")
    
    # --- 1. Generate Difference Table ---
    diff_table_df = EvenDifference(points)

    # --- 2. Select Coefficients (Bessel Path) ---
    C_coeffs = []
    for i in range(n + 1):
        order_col = f'Order {i}' if i > 0 else 'y_i'
        
        if i % 2 == 0: # Even order: C_{2k} = (d^{2k} y_{-k} + d^{2k} y_{-(k-1)}) / 2
            k = i // 2
            c1_idx = x0_index - k
            c2_idx = x0_index - k + 1
            if c1_idx < 0 or c2_idx >= len(points):
                raise IndexError(f"Cannot access indices {c1_idx}, {c2_idx} for order {i}.")
            
            c1 = diff_table_df[order_col].iloc[c1_idx]
            c2 = diff_table_df[order_col].iloc[c2_idx]
            C_coeffs.append((c1 + c2) / 2.0)
            
        else: # Odd order: C_{2k+1} = d^{2k+1} y_{-k}
            k = (i - 1) // 2
            c_idx = x0_index - k
            if c_idx < 0 or c_idx >= len(points):
                raise IndexError(f"Cannot access index {c_idx} for order {i}.")
            C_coeffs.append(diff_table_df[order_col].iloc[c_idx])

    # --- 3. Build Polynomial P(t) ---
    steps_data = []
    N_coeffs_total = np.zeros(n + 1, dtype=float)
    
    # B_i(t) basis polynomials
    B_coeffs_list = []
    
    # B_0(t) = 1
    B_0 = np.array([1.0])
    B_coeffs_list.append(B_0)
    
    # B_1(t) = (t - 0.5)
    B_1 = np.array([-0.5, 1.0]) # [const, t]
    B_coeffs_list.append(B_1)

    for i in range(2, n + 1):
        if i % 2 == 0: # Even: B_{2k} = B_{2k-2} * (t+k-1)(t-k)
            k = i // 2
            # (t - k)
            term1 = np.array([-k, 1.0])
            # (t + k - 1)
            term2 = np.array([k - 1, 1.0])
            # (t^2 - t - k(k-1))
            new_factor = np.convolve(term1, term2)
            # B_{2k-2} is at index i-2
            B_i = np.convolve(B_coeffs_list[i-2], new_factor)
        else: # Odd: B_{2k+1} = B_{2k} * (t - 0.5)
            # B_{2k} is at index i-1
            B_i = np.convolve(B_coeffs_list[i-1], [-0.5, 1.0])
        
        B_coeffs_list.append(B_i)

    # Calculate final polynomial
    for i in range(n + 1):
        D_i = C_coeffs[i] / math.factorial(i)
        B_i = B_coeffs_list[i]
        
        # N_i(t) = D_i * B_i(t)
        Ni_coeffs = D_i * B_i
        
        # Add to total polynomial (pad with zeros)
        N_coeffs_total[:len(Ni_coeffs)] += Ni_coeffs

        # Store intermediate steps for printing
        steps_data.append({
            'i': i,
            'Difference Coeff (C_i)': C_coeffs[i],
            'D_i = C_i / i!': D_i,
            'B_i(t) Coeffs (low->high)': B_i.tolist(),
            'N_i(t) Coeffs (low->high)': Ni_coeffs.tolist()
        })

    step_pd = pd.DataFrame(steps_data)
    coeff_pd = pd.DataFrame({
        'Degree (t)': np.arange(n + 1),
        'Coeff (low->high)': N_coeffs_total
    })

    return step_pd, coeff_pd

## Result

In [4]:
# 1. Define the data points from the image
points = [ (5, 3.9931011), 
     (5.1, 4.1278568), 
     (5.2, 4.2723258), 
     (5.3, 4.426064), 
     (5.4, 4.5885338), 
     (5.5, 4.7591118), 
     (5.6, 4.93709), 
     (5.7, 5.1216967) ]

# 2. Set the central node index
# 9 points (indices 0-8), the center is index 4
x0_index = 3
x0_val = points[x0_index][0]
h = points[1][0] - points[0][0]

In [5]:
print("--- Generated Finite Difference Table ---")
df = EvenDifference(points)

df.style

--- Generated Finite Difference Table ---


,x_i,y_i,Order 1,Order 2,Order 3,Order 4,Order 5,Order 6,Order 7
0,5.000000,3.993101,0.134756,0.009713,-0.000444,-0.000094,0.000008,-0.000007,0.000026
1,5.100000,4.127857,0.144469,0.009269,-0.000538,-0.000086,0.000001,0.000020,nan
2,5.200000,4.272326,0.153738,0.008732,-0.000623,-0.000085,0.000021,nan,nan
3,5.300000,4.426064,0.162470,0.008108,-0.000708,-0.000064,nan,nan,nan
4,5.400000,4.588534,0.170578,0.007400,-0.000772,nan,nan,nan,nan
5,5.500000,4.759112,0.177978,0.006628,nan,nan,nan,nan,nan
6,5.600000,4.937090,0.184607,nan,nan,nan,nan,nan,nan
7,5.700000,5.121697,nan,nan,nan,nan,nan,nan,nan


In [6]:
# 3. Calculate the Bessel polynomial
step_df, final_coeff_df = bessel_interpolation(points, x0_index)

print("\n--- Polynomial Construction Steps (Bessel) ---")
step_df.style


--- Polynomial Construction Steps (Bessel) ---


,i,Difference Coeff (C_i),D_i = C_i / i!,B_i(t) Coeffs (low->high),N_i(t) Coeffs (low->high)
0,0,4.507299,4.507299,[1.0],[4.5072989]
1,1,0.162470,0.162470,"[-0.5, 1.0]","[-0.08123490000000011, 0.16246980000000022]"
2,2,0.008420,0.004210,"[0.0, -1.0, 1.0]","[0.0, -0.004209949999999907, 0.004209949999999907]"
3,3,-0.000623,-0.000104,"[0.0, 0.5, -1.5, 1.0]","[-0.0, -5.195000000002281e-05, 0.00015585000000006843, -0.00010390000000004562]"
4,4,-0.000085,-0.000004,"[0.0, 2.0, -1.0, -2.0, 1.0]","[-0.0, -7.099999999963617e-06, 3.5499999999818086e-06, 7.099999999963617e-06, -3.5499999999818086e-06]"
5,5,0.000001,0.000000,"[0.0, -1.0, 2.5, 0.0, -2.5, 1.0]","[0.0, -1.0000000005838671e-08, 2.500000001459668e-08, 0.0, -2.500000001459668e-08, 1.0000000005838671e-08]"
6,6,0.000007,0.000000,"[0.0, -12.0, 4.0, 15.0, -5.0, -3.0, 1.0]","[0.0, -1.0999999997910828e-07, 3.666666665970276e-08, 1.3749999997388537e-07, -4.5833333324628455e-08, -2.749999999477707e-08, 9.16666666492569e-09]"
7,7,0.000026,0.000000,"[0.0, 6.0, -14.0, -3.5, 17.5, -3.5, -3.5, 1.0]","[0.0, 3.1190476187540185e-08, -7.277777777092709e-08, -1.8194444442731773e-08, 9.097222221365886e-08, -1.8194444442731773e-08, -1.8194444442731773e-08, 5.1984126979233636e-09]"


## Further Testing

In [7]:
#Horner Test
def synthetic_division(a, c):
    """
    Perform synthetic division for polynomial p(x) with coefficients a,
    evaluated at x = c.

    Parameters:
        a (list[float]): coefficients of p(x) from highest to lowest degree
        c (float): the value to evaluate p(c)

    Returns:
        df (pd.DataFrame): table with columns [i, a_i, b_i*c, b_i]
        p_c (float): value of p(c)
        q_coeff (list[float]): coefficients of q(x) = (p(x) - p(c)) / (x - c)
    """

    n = len(a) - 1
    b = [0.0] * (n + 1)
    bc_values = [""] * (n + 1)

    b[n] = a[n]
    for i in range(n - 1, -1, -1):
        b[i] = a[i] + c * b[i + 1]
        bc_values[i + 1] = b[i + 1] * c

    # Prepare table (i from n to 0)
    df = pd.DataFrame({
        "i": list(range(n, -1, -1)),
        "a_i": [a[i] for i in range(n, -1, -1)],
        "b_i*c": [bc_values[i] for i in range(n, -1, -1)],
        "b_i = a_i + b_(i+1)*c": [b[i] for i in range(n, -1, -1)]
    })

    p_c = b[0]
    q_coeff = b[1:]
    return df, p_c, q_coeff, b

def all_derivatives(a, c):
    """
    Compute all derivatives p^(i)(c) using repeated Horner division
    and display in transposed table format.
    """
    coeffs = a.copy()
    degree = len(a) - 1
    results = []
    b0_list = []
    derivative_list = []

    # Perform repeated synthetic division
    for i in range(degree + 1):
        df, b0, next_coeff, b_all = synthetic_division(coeffs, c)
        results.append(b_all)
        b0_list.append(b0)
        derivative_list.append(b0 * math.factorial(i))
        coeffs = next_coeff
        if len(coeffs) == 0:
            break

    # Pad b_i lists for equal column length
    max_len = max(len(b) for b in results)
    for b in results:
        b.extend([None] * (max_len - len(b)))

    # Create DataFrame horizontally
    df = pd.DataFrame(results).T
    df.columns = [f"i={i}" for i in range(len(results))]

    # Insert first column for original a coefficients
    a_col = a + [None] * (df.shape[0] - len(a))
    df.insert(0, "a_i", a_col)

    # Add b_0 and p^(i)(c) rows
    df.loc["b_0"] = [None] + b0_list
    df.loc["p^(i)(c)"] = [None] + derivative_list

    # Add a row on top showing the value of c
    df.loc["c"] = [c] + [None] * (df.shape[1] - 1)
    df = df.loc[["c"] + [idx for idx in df.index if idx != "c"]]  # Move row to top

    return df

In [8]:
coeff_list = final_coeff_df['Coeff (low->high)'].tolist()

x_val = 5.63
t_val = (x_val - x0_val) / h
df2 = all_derivatives(coeff_list, t_val)
df2.style

,a_i,i=0,i=1,i=2,i=3,i=4,i=5,i=6,i=7
c,3.300000,nan,nan,nan,nan,nan,nan,nan,nan
0,4.426064,4.991812,0.183377,0.003195,-0.000132,0.000001,0.000001,0.000000,0.000000
1,0.158201,0.171439,0.003618,-0.000128,-0.000001,0.000001,0.000000,0.000000,nan
2,0.004369,0.004012,-0.000119,-0.000003,0.000000,0.000000,0.000000,nan,nan
3,-0.000097,-0.000108,-0.000003,0.000000,0.000000,0.000000,nan,nan,nan
4,-0.000004,-0.000004,0.000000,0.000000,0.000000,nan,nan,nan,nan
5,-0.000000,-0.000000,0.000000,0.000000,nan,nan,nan,nan,nan
6,-0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan
7,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
b_0,nan,4.991812,0.183377,0.003195,-0.000132,0.000001,0.000001,0.000000,0.000000
